<a href="https://colab.research.google.com/github/Parzival-Prime/Fish-vs-Cat-Classifier/blob/main/Convolutional_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

In [ ]:
# !kaggle datasets download zuraiz/fish-vs-cats-imagenet-subdataset

Dataset URL: https://www.kaggle.com/datasets/zuraiz/fish-vs-cats-imagenet-subdataset
License(s): unknown
  0% 0.00/67.7M [00:00<?, ?B/s]
100% 67.7M/67.7M [00:00<00:00, 1.37GB/s]


In [ ]:
# import zipfile
# zipref = zipfile.ZipFile('fish-vs-cats-imagenet-subdataset.zip', 'r')
# zipref.extractall()
# zipref.close()

### Prepare Dataset

In [ ]:
import torch
import torchvision
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
import os

In [ ]:
train_data_path = os.path.join('Fish-vs-Cats', 'train')
test_data_path = os.path.join('Fish-vs-Cats', 'test')
val_data_path = os.path.join('Fish-vs-Cats', 'val')

In [ ]:
transforms = v2.Compose([
    v2.Resize((64, 64)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225])
])

In [ ]:
from PIL import Image, UnidentifiedImageError
class SafeImageFolder(torchvision.datasets.ImageFolder):
  def __getitem__(self, index):
    path, target = self.samples[index]
    try:
      sample = self.loader(path)
    except UnidentifiedImageError:
      return self.__getitem__((index + 1) % len(self.samples))
    if self.transform is not None:
      sample = self.transform(sample)
    return sample, target

In [ ]:
train_data = SafeImageFolder(train_data_path, transforms)
test_data = SafeImageFolder(test_data_path, transforms)
val_data = SafeImageFolder(val_data_path, transforms)

In [ ]:
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

## Creating Neural Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cuda


In [ ]:
class CNNNet(nn.Module):
  def __init__(self, num_classes=2):
    super().__init__()
    self.features = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),

        nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),

        nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1),
        nn.ReLU(),

        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
        nn.ReLU(),

        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
    self.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=256 * 6 * 6, out_features=4096),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(in_features=4096, out_features=4096),
        nn.ReLU(),
        nn.Linear(in_features=4096, out_features=num_classes),
    )

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

In [ ]:
model = CNNNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 100

## Train Function

In [ ]:
def train(model, train_loader, val_loader, optimizer, criterion, epochs, device):
  for epoch in range(epochs):
    model.train()
    training_loss = 0
    val_loss = 0
    train_correct = 0
    train_total = 0
    val_correct = 0
    val_total = 0

    for X_train, y_train in train_loader:
      X_train, y_train = X_train.to(device), y_train.to(device)
      optimizer.zero_grad()
      y_pred = model(X_train)
      loss = criterion(y_pred, y_train)
      loss.backward()
      optimizer.step()
      training_loss += loss.data.item()
      correct = torch.eq(torch.max(F.softmax(y_pred, dim=1), dim=1)[1], y_train).view(-1)
      train_correct += torch.sum(correct).item()
      train_total += correct.shape[0]
    training_loss /= len(train_loader)

    model.eval()
    for X_val, y_val in val_loader:
      X_val, y_val = X_val.to(device), y_val.to(device)
      optimizer.zero_grad()
      y_pred = model(X_val)
      loss = criterion(y_pred, y_val)
      loss.backward()
      optimizer.step()
      val_loss = loss.data.item()
      correct = torch.eq(torch.max(F.softmax(y_pred, dim=1), dim=1)[1], y_val).view(-1)
      val_correct += torch.sum(correct).item()
      val_total += correct.shape[0]
    val_loss /= len(val_loader)

    print(f'Epoch: {epoch}, Training Loss: {training_loss:.2f}, Training_accuracy: {((train_correct / train_total) * 100):.2f},  Validation Loss: {val_loss:.2f}, val_accuracy = {((val_correct / val_total) * 100):.2f}%')


In [ ]:
dummy_input = (torch.randn(1, 3, 64, 64)).to(device)  # adjust based on your input image
output = model.features(dummy_input)
print(output.shape)

torch.Size([1, 256, 1, 1])


In [ ]:
train(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=epochs,
    device=device
)

Epoch: 0, Training Loss: 1.10, Training_accuracy: 50.99,  Validation Loss: 0.34, val_accuracy = 67.23%
Epoch: 1, Training Loss: 0.68, Training_accuracy: 57.18,  Validation Loss: 0.33, val_accuracy = 70.59%
Epoch: 2, Training Loss: 0.62, Training_accuracy: 65.97,  Validation Loss: 0.27, val_accuracy = 84.03%
Epoch: 3, Training Loss: 0.53, Training_accuracy: 74.50,  Validation Loss: 0.27, val_accuracy = 79.83%
Epoch: 4, Training Loss: 0.47, Training_accuracy: 80.07,  Validation Loss: 0.18, val_accuracy = 85.71%
Epoch: 5, Training Loss: 0.47, Training_accuracy: 79.58,  Validation Loss: 0.23, val_accuracy = 60.50%
Epoch: 6, Training Loss: 0.49, Training_accuracy: 77.72,  Validation Loss: 0.18, val_accuracy = 78.99%
Epoch: 7, Training Loss: 0.47, Training_accuracy: 80.69,  Validation Loss: 0.17, val_accuracy = 86.55%
Epoch: 8, Training Loss: 0.42, Training_accuracy: 81.56,  Validation Loss: 0.10, val_accuracy = 89.92%
Epoch: 9, Training Loss: 0.45, Training_accuracy: 80.07,  Validation Loss

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
from PIL import Image
labels = ['cat', 'fish']

In [ ]:
image = Image.open('cat.jpg')
image = transforms(image).unsqueeze(0)
image = image.to(device)
prediction = model(image)
prediction = prediction.argmax()
print(labels[prediction])

cat


### Saving Model

In [ ]:
torch.save(model, "fish-vs-cal-classifier.pkl")

In [ ]:
torch.save(model.state_dict(),"fish-vs-cal-classifier.pth")